In [167]:
class PlugBoard():
    def generate_plugs():#wow, much random
        return {0:1,2:3,4:5,6:7,17:21,13:24,10:19,11:14,18:22,8:20}
    
    def valid_plugs(self):
        letters = []
        counter_plug = 0
        counter_self_plug = 0
        for k, v in self.plugs.items():
            if(k in letters):
                return False, counter_plug
            letters.append(k)
            if(k==v):
                counter_self_plug+=1
                continue
            else:
                counter_plug+=1 
            if(v in letters):
                return False, counter_plug
            letters.append(v)
        
        return counter_plug<=10, counter_plug
    
    def __init__(self, plugs, validate_plugs):
        self.plugs = plugs
        self.inv_plugs = {v: k for k, v in plugs.items()}
        if(validate_plugs and not self.valid_plugs()[0]):
            raise ValueError("Bad plugs")
            
    def run(self,val):
        if(val in self.plugs.keys()):
            #print("Plugs: Turn {} into {}".format(chr(val+65),chr(self.plugs[val]+65)))
            return self.plugs[val]
        if(val in self.inv_plugs.keys()):
            #print("Plugs: Turn {} into {}".format(chr(val+65),chr(self.inv_plugs[val]+65)))   
            return self.inv_plugs[val]
        return val
    
    def get_plugged_letters(self):#gets list of letters thet have plug plugged into them
        return list(set(list(self.plugs.keys())+list(self.plugs.values())))

In [168]:
class Reflector():
    def __init__(self):
        wiring = "EJMZALYXVBWFCRQUONTSPIKHGD"
        self.wirinig = []
        for idx,letter in enumerate(wiring):
            self.wirinig.append(ord(letter)-65)
            
    def run(self, val):
        #print("Reflector: Turn {} into {}".format(chr(val+65),chr(self.wirinig[val]+65)))
        return self.wirinig[val]

In [169]:
class Rotor():
    def __init__(self, typ):
        self.position = 0
        self.type = typ
        if(self.type == 0):
            wirinig = "EKMFLGDQVZNTOWYHXUSPAIBRCJ"
        if(self.type == 1):
            wirinig = "AJDKSIRUXBLHWTMCQGZNPYFVOE"
        if(self.type == 2):
            wirinig = "BDFHJLCPRTXVZNYEIWGAKMUSQO"
        if(self.type == 3):
            wirinig = "ESOVPZJAYQUIRHXLNFTGKDCMWB"
        if(self.type == 4):
            wirinig = "VZBRGITYUPSDNHLXAWMJQOFECK"
        self.wirinig = []
        for idx,letter in enumerate(wirinig):
            self.wirinig.append(ord(letter)-65)
            
    def run(self, val, forward):
        # Electricity passing through rotor
        if(forward):
            val = (val+self.position)%26
            #print("Rotor: Turn {} into {}".format(chr(val+65),chr(self.wirinig[val]+65)))
            return self.wirinig[val]
        else:
            #print("Reverse: Turn {} into {}".format(chr(val+65) ,chr((self.wirinig.index(val)-self.position)%26+65)))
            val = (self.wirinig.index(val)-self.position)%26
            return val
        
            

In [240]:
class Enigma():
    def __init__(self, rotor_type_1, rotor_type_2, rotor_type_3, plugs, validate_plugs):
        self.rotor1 = Rotor(rotor_type_1)
        self.rotor2 = Rotor(rotor_type_2)
        self.rotor3 = Rotor(rotor_type_3)
        self.reflector = Reflector()
        self.plug_board = PlugBoard(plugs, validate_plugs)
       
    def enigmify(self,text):
        text = text.upper()
        enigmified_text = ""
        for letter in text:
            letter = ord(letter)-65
            if(letter<0 or letter>25):
                raise ValueError("Bad symbol")
            letter = self.plug_board.run(letter)
            letter = self.run_rotors(letter)
            letter = self.plug_board.run(letter)
            enigmified_text = enigmified_text+chr(letter+65)
            self.move_rotors()
        return enigmified_text
    
    def enigmify_letter(self, letter):
        letter = self.plug_board.run(letter)
        letter = self.run_rotors(letter)
        letter = self.plug_board.run(letter)
        self.move_rotors()
        return letter
    def set_rotor_positions(self, pos1, pos2, pos3):
        self.rotor1.position = pos1
        self.rotor2.position = pos2
        self.rotor3.position = pos3
    
    def run_rotors(self,val):
        val = self.rotor1.run(val,forward=True)
        val = self.rotor2.run(val,forward=True)
        val = self.rotor3.run(val,forward=True)
        val = self.reflector.run(val)
        val = self.rotor3.run(val,forward=False)
        val = self.rotor2.run(val,forward=False)
        val = self.rotor1.run(val,forward=False)
        return val
    
    def move_rotors(self):
        self.rotor1.position +=1
        if(self.rotor1.position>25):
            self.rotor1.position = 0
            self.rotor2.position +=1
            if(self.rotor2.position>25):
                self.rotor2.position = 0
                self.rotor3.position +=1
                if(self.rotor3.position>25):
                    self.rotor3.position = 0

In [241]:
plugs = PlugBoard.generate_plugs()

In [242]:
enigma = Enigma(0, 1, 2, plugs, True)

In [243]:
enigma.plug_board.get_plugged_letters()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 17, 18, 19, 20, 21, 22, 24]

In [228]:
enigma.set_rotor_positions(0,0,10)
enigma.enigmify("LOREMIPSUMDOLORSITAMETCONSECTETURADIPISCINGELITCRASNEQUEIPSUMINTERDUMEUCURSUSVELPHARETRABLANDITSAPIENINTEGERELEIFENDSEDELITIMPERDIETVENENATISMORBIVELTORTOREFFICITURTINCIDUNTARCUAUCTORVESTIBULUMDUIQUISQUEPHARETRALACUSDIAMSITAMETALIQUETNUNCSUSCIPITQUISMORBIEUMIVITAEJUSTOEUISMODULLAMCORPERFUSCESCELERISQUENISIVELODIODIGNISSIMFERMENTUMSEDSEMPERVEHICULASEMEUSOLLICITUDINDONECVITAEDIGNISSIMDOLORINMAXIMUSDIGNISSIMLECTUSQUISPORTTITORFUSCEVITAEMISEDURNAMATTISCONVALLIS")

'VIXRSWHDWCODFRPCFOYBCNAAPYKNAVSAWYGCIUUNHVDBRYEHBFTKDUNRFHZMCCLQXOWWPUWHOOVEYGMNIPUEUPSVQFMAYCIGPFVPZZKKRRPAOGLKOLMLTCFSUCLHUUFKCYLWJWSGWLAHPDAUVCBTGSIMCNYZHZZAVMVCFGXMWFYFPRAUQHTUJCJYLVLPXPGTEPYCLRGJIEBDUFKCVNQGQAJOJJDEIPMUVUPMMKPXISVDRKKDEOCUJJZGNYRIPORDXHMHNROQZNVCBVIEGUGRFZPEYZTGEKPYXGBIEWRWOEFJKMZFZDOSAENNLRQHHODJTGDIIYIZFDQSJYVFWOFQZGMKGYAWLCIZGMJAZRYQYVKNVHRTEQSFAGLUAMSZEXRRLJCAFWXDDOCNQRHBFOKLVIPUUNBPWNHDXKOPPGRJHLIAOIZUKEOFZLEDFVOXDXKTVRJAPYKYWMTVA'

In [229]:
enigma.set_rotor_positions(0,0,10)
enigma.enigmify('VIXRSWHDWCODFRPCFOYBCNAAPYKNAVSAWYGCIUUNHVDBRYEHBFTKDUNRFHZMCCLQXOWWPUWHOOVEYGMNIPUEUPSVQFMAYCIGPFVPZZKKRRPAOGLKOLMLTCFSUCLHUUFKCYLWJWSGWLAHPDAUVCBTGSIMCNYZHZZAVMVCFGXMWFYFPRAUQHTUJCJYLVLPXPGTEPYCLRGJIEBDUFKCVNQGQAJOJJDEIPMUVUPMMKPXISVDRKKDEOCUJJZGNYRIPORDXHMHNROQZNVCBVIEGUGRFZPEYZTGEKPYXGBIEWRWOEFJKMZFZDOSAENNLRQHHODJTGDIIYIZFDQSJYVFWOFQZGMKGYAWLCIZGMJAZRYQYVKNVHRTEQSFAGLUAMSZEXRRLJCAFWXDDOCNQRHBFOKLVIPUUNBPWNHDXKOPPGRJHLIAOIZUKEOFZLEDFVOXDXKTVRJAPYKYWMTVA')

'LOREMIPSUMDOLORSITAMETCONSECTETURADIPISCINGELITCRASNEQUEIPSUMINTERDUMEUCURSUSVELPHARETRABLANDITSAPIENINTEGERELEIFENDSEDELITIMPERDIETVENENATISMORBIVELTORTOREFFICITURTINCIDUNTARCUAUCTORVESTIBULUMDUIQUISQUEPHARETRALACUSDIAMSITAMETALIQUETNUNCSUSCIPITQUISMORBIEUMIVITAEJUSTOEUISMODULLAMCORPERFUSCESCELERISQUENISIVELODIODIGNISSIMFERMENTUMSEDSEMPERVEHICULASEMEUSOLLICITUDINDONECVITAEDIGNISSIMDOLORINMAXIMUSDIGNISSIMLECTUSQUISPORTTITORFUSCEVITAEMISEDURNAMATTISCONVALLIS'

Шось тре дєлать

Алгорітм, 

Беремо перші сеттінги роторів

Берем слово яке там точно є

Підставляємо його по рядку тексту, в поісках можливих позицій(буква не може бути собою)

Робимо гіпотезу, які букви на плагбоарді з'єднані

Робимо дедукції з гіпотепи #як дєлать дедукції з гіпотези

Якшо гіпотеза суперечаща, записуєм суперечності, і робим наступну гіпотезу

Якшо всі гіпотези до якоїсь букви суперечащі, значить в нас погані сеттінги роторів

Берем наступні сеттінги роторів і повтойряємо з рядку 5

Якщо знайшли робочу гіпотезу, записуємо її та всі дедукції

Профіт

In [246]:
craker = Cracker('VIXRSWHDWCODFRPCFOYBCNAAPYKNAVSAWYGCIUUNHVDBRYEHBFTKDUNRFHZMCCLQXOWWPUWHOOVEYGMNIPUEUPSVQFMAYCIGPFVPZZKKRRPAOGLKOLMLTCFSUCLHUUFKCYLWJWSGWLAHPDAUVCBTGSIMCNYZHZZAVMVCFGXMWFYFPRAUQHTUJCJYLVLPXPGTEPYCLRGJIEBDUFKCVNQGQAJOJJDEIPMUVUPMMKPXISVDRKKDEOCUJJZGNYRIPORDXHMHNROQZNVCBVIEGUGRFZPEYZTGEKPYXGBIEWRWOEFJKMZFZDOSAENNLRQHHODJTGDIIYIZFDQSJYVFWOFQZGMKGYAWLCIZGMJAZRYQYVKNVHRTEQSFAGLUAMSZEXRRLJCAFWXDDOCNQRHBFOKLVIPUUNBPWNHDXKOPPGRJHLIAOIZUKEOFZLEDFVOXDXKTVRJAPYKYWMTVA',"LOREMIPSUMDOLORSITAMET")

craker.crack()

ROTORS: 0 1 2
Testing rotor positions: 0 0 X
Possible configuration: rotors 0 1 2 rotor positions 0 0 10
 plug board {0: 1, 13: 24, 12: 12, 10: 19, 22: 18, 20: 8, 2: 3, 11: 14, 4: 5, 21: 17, 15: 15, 23: 23, 6: 7}
Testing rotor positions: 0 1 X
Testing rotor positions: 0 2 X


KeyboardInterrupt: 

In [245]:
class Cracker():
    def check_crib(self, pos):
        #check whether crib can fit to message at given position
        for i in range(len(self.crib)):
            if(self.crib[i] == self.msg[pos+i]):
                return False
        return True
    
    def __init__(self, msg, crib):       
        self.msg = msg
        self.converted_msg = [ord(char)-65 for char in msg]
        self.crib = crib
        self.converted_crib = [ord(char)-65 for char in crib]
        self.working_positions = []
        for pos in range(len(msg)-len(crib)+1):
            if(self.check_crib(pos)):
                self.working_positions.append(pos)
        self.enigma = Enigma(0,1,2,{}, False)
        
    def deduce(self, pos, letter1, letter2,pos1,pos2,pos3):
        #set default plugs on current iteration
        self.enigma.plug_board.plugs = {letter1:letter2}
        self.enigma.plug_board.inv_plugs = {letter2: letter1}
        
        for _ in range(26):
            #set tested rotor positions for deduction iteration
            self.enigma.set_rotor_positions(pos1,pos2,pos3)
            
            for i in range(len(self.crib)):
                #check whether a letter is deducable
                use_crib_letter = False
                if(self.converted_msg[pos+i] not in self.enigma.plug_board.get_plugged_letters()):
                    use_crib_letter = True
                    if(self.converted_crib[i] not in self.enigma.plug_board.get_plugged_letters()):
                        self.enigma.move_rotors()
                        continue

                #make a deduction
                letter = self.enigma.enigmify_letter(self.converted_crib[i] if use_crib_letter else self.converted_msg[pos+i])
                if(letter not in self.enigma.plug_board.get_plugged_letters()):
                    self.enigma.plug_board.plugs[letter] = self.converted_msg[pos+i] if use_crib_letter else self.converted_crib[i]
                    self.enigma.plug_board.inv_plugs[self.converted_msg[pos+i] if use_crib_letter else self.converted_crib[i]] = letter
                #exit condition: either plugboard is invalid or complete
                if(not self.enigma.plug_board.valid_plugs()[0] or self.enigma.plug_board.valid_plugs()[1]==10):
                    break

        return self.enigma.plug_board.valid_plugs()[0]
    
    def crack(self):
        for rotor1 in range(5):
            for rotor2 in range(5):
                if(rotor2==rotor1):continue
                for rotor3 in range(5):
                    if(rotor3==rotor2 or rotor3==rotor1):continue
                    self.enigma = Enigma(rotor1,rotor2,rotor3,{},False)
                    print("ROTORS: {} {} {}".format(rotor1,rotor2,rotor3))
                    for working_position in self.working_positions:
                        for pos1 in range(26):
                            for pos2 in range(26):
                                print("Testing rotor positions: {} {} X".format(pos1,pos2))
                                for pos3 in range(26):

                                    letter_ban = []

                                    for letter1 in range(26):
                                        for letter2 in range(26):

                                            if((letter1,letter2) in letter_ban or (letter2,letter1) in letter_ban):
                                                continue

                                            self.enigma.set_rotor_positions(pos1,pos2,pos3)
                                            for _ in range(working_position):
                                                self.enigma.move_rotors()

                                            if(self.deduce(working_position, letter1, letter2,pos1,pos2,pos3)):                          
                                                if(self.enigma.plug_board.valid_plugs()[1]==10):
                                                    self.enigma.set_rotor_positions(pos1,pos2,pos3)
                                                    decoded_msg = self.enigma.enigmify(self.msg)
                                                    
                                                    if(self.crib in decoded_msg):
                                                        print("Possible configuration: rotors {} {} {} rotor positions {} {} {}\n plug board {}".format(rotor1,rotor2,rotor3,pos1,pos2,pos3,self.enigma.plug_board.plugs))

                                            letter_ban.extend(self.enigma.plug_board.plugs.items())
                                            letter_ban.extend(self.enigma.plug_board.inv_plugs.items())
